In [7]:
#module 5 lesson 3 "relax and hold steady"
#import libraries
import numpy
from matplotlib import pyplot
%matplotlib inline
from helper import laplace_solution, l2_norm, plot_3d

In [2]:
# Set some parameters
nx, ny = 128, 128  # number of points in the x/y direction
Lx, Ly = 5.0, 5.0  # domain length in the x/y direction
dx = Lx / (nx - 1)  # grid spacing in x direction
dy = Ly / (ny - 1)  # grid spacing in y direction

# Create the gridline locations.
x = numpy.linspace(0.0, Lx, num=nx)
y = numpy.linspace(0.0, Ly, num=ny)

# Set the initial conditions.
p0 = numpy.zeros((ny, nx))
p0[-1, :] = numpy.sin(1.5 * numpy.pi * x / Lx)

In [5]:
def laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-6):
    p = p0.copy()
    diff = rtol + 1.0  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        # Update the solution at interior points.
        p[1:-1, 1:-1] = 0.25 * (pn[1:-1, :-2] + pn[1:-1, 2:] +
                                pn[:-2, 1:-1] + pn[2:, 1:-1])
        # Apply 2nd-order Neumann condition (zero-gradient)
        # at the right boundary.
        p[1:-1, -1] = 0.25 * (2.0 * pn[1:-1, -2] +
                              pn[2:, -1] + pn[:-2, -1])
        # Compute the relative L2-norm of the difference.
        diff = l2_norm(p, pn)
        ite += 1
    return p, ite, diff

In [8]:
# Compute the solution using Jacobi relaxation method.
p, ites, diff = laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-8)
print(ites)
print(diff)

19993
9.998616841218672e-09


In [9]:
%%timeit
laplace_2d_jacobi(p0, maxiter=20000, rtol=1e-8)

3.76 s ± 234 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# Compute the analytical solution.
p_exact = laplace_solution(x, y, Lx, Ly)

# Compute the relative L2-norm of the error.
l2_norm(p, p_exact)

6.173551335287984e-05

In [11]:
def laplace_2d_gauss_seidel(p0, maxiter=20000, rtol=1e-6):
    ny, nx = p0.shape
    p = p0.copy()
    diff = rtol + 1.0  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        # Update the solution at interior points.
        for j in range(1, ny - 1):
            for i in range(1, nx - 1):
                p[j, i] = 0.25 * (p[j, i - 1] + p[j, i + 1] +
                                  p[j - 1, i] + p[j + 1, i])
        # Apply 2nd-order Neumann condition (zero-gradient)
        # at the right boundary.
        for j in range(1, ny - 1):
            p[j, -1] = 0.25 * (2.0 * p[j, -2] +
                               p[j - 1, -1] + p[j + 1, -1])
        # Compute the relative L2-norm of the difference.
        diff = l2_norm(p, pn)
        ite += 1
    return p, ite, diff

In [13]:
import numba
from numba import jit

In [18]:
def fib_it(n):
    a, b = 1, 1
    for i in range(n - 2):
        a, b = b, a + b
    return b

In [19]:
%%timeit
fib_it(500000)

3.4 s ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
@jit
def fib_it(n):
    a, b = 1, 1
    for i in range(n - 2):
        a, b = b, a + b
    return b

In [21]:
%%timeit
fib_it(500000)

205 µs ± 4.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
#Successive Over Relaxation
def laplace_2d_sor(p0, omega, maxiter=20000, rtol=1e-6):
    ny, nx = p0.shape
    p = p0.copy()
    conv = []  # convergence history
    diff = rtol + 1.0  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        # Update the solution at interior points.
        for j in range(1, ny - 1):
            for i in range(1, nx - 1):
                p[j, i] = ((1.0 - omega) * p[j, i] +
                           omega * 0.25 *(p[j, i - 1] + p[j, i + 1] +
                                          p[j - 1, i] + p[j + 1, i]))
        # Apply 2nd-order Neumann condition (zero-gradient)
        # at the right boundary.
        for j in range(1, ny - 1):
            p[j, -1] = 0.25 * (2.0 * p[j, -2] +
                               p[j - 1, -1] + p[j + 1, -1])
        # Compute the relative L2-norm of the difference.
        diff = numpy.sqrt(numpy.sum((p - pn)**2) / numpy.sum(pn**2))
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [23]:
omega = 1.0
p, ites, conv_sor = laplace_2d_sor(p0, omega,
                                   maxiter=20000, rtol=1e-8)
print(ites)
print(conv_sor)

KeyboardInterrupt: 

In [ ]:
omega = 1.5
p, ites, conv_sor = laplace_2d_sor(p0, omega,
                                   maxiter=20000, rtol=1e-8)
print(ites)
print(conv_sor)

In [ ]:
#tune the SOR
omega = 2.0 / (1.0 + numpy.pi / nx)
p, ites, conv_sor = laplace_2d_sor(p0, omega, maxiter=20000, rtol=1e-8)
print(ites)
print(conv_sor)